In [17]:
from ast import literal_eval
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/kexinhuang/Downloads/TDC Leaderboard Submission Form.csv')

In [18]:
df = df.rename(columns = {
    df.columns.values[0]: 'Time',
    df.columns.values[1]: 'Name',
    df.columns.values[2]: 'Email',
    df.columns.values[3]: 'Group',
    df.columns.values[4]: 'Model',
    df.columns.values[5]: 'Paper_URL',
    df.columns.values[6]: 'Code_URL',
    df.columns.values[7]: 'Result',
    df.columns.values[8]: 'Hardware',
    df.columns.values[9]: '#Params'    
})
df = df.drop(df.columns.values[10], axis = 1)

In [19]:
df['Result'] = df.Result.apply(literal_eval)

In [20]:
metrics_direction = {
    'mae': 'desc',
    'roc-auc': 'asc', 
    'pr-auc': 'asc',
    'pcc': 'asc'
}

In [29]:
def retrieve_mean(x, categories):
    l = []
    for i in categories:
        l.append(x[i][0])
    return np.array(l)

## ADMET Group

In [25]:
df_admet = df[df.Group == 'ADMET Group']

In [26]:
admet_metrics = {'caco2_wang': 'mae',
            'hia_hou': 'roc-auc',
            'pgp_broccatelli': 'roc-auc', 
            'bioavailability_ma': 'roc-auc',
            'lipophilicity_astrazeneca': 'mae',
            'solubility_aqsoldb': 'mae',
            'bbb_martins': 'roc-auc',
            'ppbr_az': 'mae',
            'vdss_lombardo': 'pcc',
            'cyp2c9_veith': 'pr-auc',
            'cyp2d6_veith': 'pr-auc',
            'cyp3a4_veith': 'pr-auc',
            'cyp2c9_substrate_carbonmangels': 'pr-auc',
            'cyp3a4_substrate_carbonmangels': 'roc-auc',
            'cyp2d6_substrate_carbonmangels': 'pr-auc',
            'half_life_obach': 'pcc',
            'clearance_hepatocyte_az': 'pcc',
            'clearance_microsome_az': 'pcc',
            'ld50_zhu': 'mae',
            'herg': 'roc-auc',
            'ames': 'roc-auc',
            'dili': 'roc-auc'
            }

In [54]:
ADMET = {
    'Absorption': ['caco2_wang', 'hia_hou', 'pgp_broccatelli', 'bioavailability_ma', 'lipophilicity_astrazeneca', 'solubility_aqsoldb'],
    'Distribution': ['bbb_martins', 'ppbr_az', 'vdss_lombardo'],
    'Metabolism': ['cyp2c9_veith', 'cyp2d6_veith', 'cyp3a4_veith', 'cyp2c9_substrate_carbonmangels', 'cyp2d6_substrate_carbonmangels', 'cyp3a4_substrate_carbonmangels'],
    'Excretion': ['half_life_obach', 'clearance_hepatocyte_az', 'clearance_microsome_az'],
    'Toxicity': ['ld50_zhu', 'herg', 'ames', 'dili']
}

In [97]:
def get_entries(category):
    
    results = np.stack(df_admet.Result.apply(lambda x: retrieve_mean(x, ADMET[category])).values)
    print(results)
    rank_dict = dict(zip(range(results.shape[0]), list([0] * results.shape[0])))
    
    for idx, i in enumerate(range(results.T.shape[0])):
        # for every benchmark
        sort_idx = np.argsort(results.T[i], axis = 0)
        if metrics_direction[admet_metrics[ADMET[category][idx]]] == 'desc':
            sort_idx = np.argsort(results.T[i], axis = 0)
        else:
            sort_idx = np.argsort(results.T[i], axis = 0)[::-1]
        
        for pos, val in enumerate(sort_idx):
            rank_dict[val] = rank_dict[val] + pos
    
    rank = [] # [rank for 1, rank for 2, ...]
    for i in range(results.shape[0]):
        rank.append(rank_dict[i])
        
    out = np.argsort(rank)
    
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    
    entries = []
    for submit_idx in range(len(df_admet)):
        temp = df_admet.iloc[submit_idx]
        html_code = '<tr> \n'
        html_code = html_code + '  <td> ' + str(idx2rank[submit_idx] + 1) + ' </td> \n'        
        html_code = html_code + '  <td> ' + temp.Model + ' </td> \n'
        html_code = html_code + '  <td><a href="mailto:' + temp.Email + '">' + temp.Name + '</a></td> \n'
        html_code = html_code + '  <td><a href="' + temp.Code_URL + '"> GitHub</a>, <a href="' + temp.Paper_URL + '">Paper </a></td> \n'
        value = temp['#Params']
        html_code = html_code + '  <td> ' + f'{value:,}' + ' </td> \n'
        for i in ADMET[category]:
            html_code = html_code + '  <td>' + format(temp.Result[i][0], '.3f') + ' <span>&#177;</span> ' + format(temp.Result[i][1], '.3f') +' </td> \n'
        html_code += '</tr>'

        entries.append(html_code)
    
    for i in range(len(out)):
        print(entries[rank2idx[i]])

In [102]:
get_entries('Toxicity')

[[0.675 0.754 0.776 0.792]
 [0.678 0.841 0.823 0.875]
 [0.649 0.736 0.794 0.832]]
<tr> 
  <td> 1 </td> 
  <td> RDKit2D + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/kexinhuang12345/DeepPurpose"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 633,409 </td> 
  <td>0.678 <span>&#177;</span> 0.003 </td> 
  <td>0.841 <span>&#177;</span> 0.020 </td> 
  <td>0.823 <span>&#177;</span> 0.011 </td> 
  <td>0.875 <span>&#177;</span> 0.019 </td> 
</tr>
<tr> 
  <td> 2 </td> 
  <td> Morgan + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/kexinhuang12345/DeepPurpose"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 1,477,185 </td> 
  <td>0.649 <span>&#177;</span> 0.019 </td> 
  <td>0.736 <span>&#177;</span> 0.023 </td> 
  <td>0.794 <span>&#1